## Programmierbeispiel

### Fallbeispiel

#### IntelliJ IDEA

* IDE für Java-Entwickler
* Fast komplett in Java geschrieben
* Großes und lang aktives Projekt

### I. Fragestellung (1/3)

* Schreibe die Frage explizit auf
* Erkläre die Anayseidee verständlich


### I. Fragestellung (2/3)

<b>Frage</b>
* Welche Quellcodedateien sind besonders komplex und änderten sich in letzter Zeit häufig?


### I. Fragestellung (3/3)
#### Umsetzungsideen
* Werkzeuge: Jupyter, Python, pandas, matplotlib
* Heuristiken:
 * "komplex": viele Quellcodezeilen
 * "ändert ... häufig": hohe Anzahl Commits
 * "in letzter Zeit": letzte 90 Tage


**Meta-Ziel:** Grundmechaniken kennenlernen.

### II. Explorative Datenanalyse
* Finde und lade mögliche Softwaredaten
* Bereinige und filtere die Rohdaten

*Wir laden einen Datenexport aus einem Git-Repository.*

In [ ]:
import pandas as pd
log = pd.read_csv("dataset/git_log_intellij.csv.gz")
log.head()

*Wir sehen uns Basisinfos über den Datensatz an.*

In [ ]:
log.info()

<b>1</b> **DataFrame** (~ programmierbares Excel-Arbeitsblatt), <b>6</b> **Series** (= Spalten), <b>1128819</b> **entries** (= Reihen)

*Wir wandeln die Zeitstempel von Texte in Objekte um.*

In [ ]:
log['timestamp'] = pd.to_datetime(log['timestamp'])
log.head()

*Wir sehen uns nur die jüngsten Änderungen an.*

In [ ]:
# use log['timestamp'].max() instead of pd.Timedelta('today') to avoid outdated data in the future
recent = log[log['timestamp'] > log['timestamp'].max() - pd.Timedelta('90 days')]
recent.head()

*Wir wollen nur Java-Code verwenden.*

In [ ]:
java = recent[recent['filename'].str.endswith(".java")].copy()
java.head() 

### III. Formale Modellierung

* Schaffe neue Sichten
* Verschneide weitere Daten

*Wir zählen die Anzahl der Änderungen je Datei.*

In [ ]:
changes = java.groupby('filename')[['sha']].count()
changes.head()

*Wir holen Infos über die Code-Zeilen hinzu...*

In [ ]:
loc = pd.read_csv("dataset/cloc_intellij.csv.gz", index_col=1)
loc.head()

*...und verschneiden diese mit den vorhandenen Daten.*

In [ ]:
hotspots = changes.join(loc[['code']]).dropna(subset=['code'])
hotspots.head()

### VI. Interpretation
* Erarbeite das Kernergebnis der Analyse heraus
* Mache die zentrale Botschaft / neuen Erkenntnisse deutlich

*Wir zeigen nur die TOP 10 Hotspots im Code an.*

In [ ]:
top10 = hotspots.sort_values(by="sha", ascending=False).head(10)
top10

### V. Kommunikation
* Transformiere die Erkenntnisse in eine verständliche Visualisierung
* Kommuniziere die nächsten Schritte nach der Analyse

*Wir erzeugen ein XY-Diagramm aus der TOP 10 Liste.*

In [ ]:
ax = top10.plot.scatter('sha', 'code');

for k, v in top10.iterrows():
    ax.annotate(k.split("/")[-1], v)

Bonus: Welche Dateien ändern sich besonders häufig im Allgemeinen?

In [ ]:
most_changes = hotspots['sha'].sort_values(ascending=False)
most_changes.head(10)

*Wir visualisieren dies mit einem einfachen Liniendiagramm.*

In [ ]:
most_changes.plot(rot=90);

### Ende der Demo